In [ ]:
# Задание 1.
# Обязательная часть
# Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

# Вам необходимо собирать только те статьи, в заголовках которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

# KEYWORDS = [‘python’, ‘парсинг’]

# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
i = 0
df = pd.DataFrame(columns = ['дата', 'заголовок', 'ссылка'])
KEYWORDS = ['python', 'парсинг','физика','двоичной', 'логики']

for page in range(1,51):
  resp = requests.get('http://habr.com/ru/all/page' + str(page) + '/')
  soup = BeautifulSoup(resp.text, 'html.parser')

  for tag in soup.find_all("a" , class_="tm-article-snippet__title-link"):
    i += 1
    name = tag.find('span').text
    # полученное название статьи проверяем на вхождение в KEYWORDS
    for word in KEYWORDS: 
      if word.lower() in list(map(lambda x: x.lower(),name.split(' '))):
        link = tag.get('href')
        date = soup.find('span', class_="tm-article-snippet__datetime-published").find('time').get('title')    
        df.loc[i] = [date,
                    name,
                    'http://habr.com' + link]
      


In [59]:
df

,дата,заголовок,ссылка
1,"2022-02-23, 17:34",Физика двоичной логики,http://habr.com/ru/company/timeweb/blog/653159/
75,"2022-02-22, 13:24",Бот-трафик и парсинг цен – взгляд со стороны в...,http://habr.com/ru/company/proto/blog/652861/
91,"2022-02-22, 09:25",Обнаружение новизны изображений с помощью Pyth...,http://habr.com/ru/post/652851/
157,"2022-02-21, 10:47",Машинное стереозрение для новичков: две камеры...,http://habr.com/ru/company/skillfactory/blog/6...
182,"2022-02-20, 01:48",Еще один способ развернуть python автотесты we...,http://habr.com/ru/post/652511/
316,"2022-02-17, 13:15",Atlas: Путешествие Dropbox от монолита на Pyth...,http://habr.com/ru/company/flant/blog/651871/
327,"2022-02-17, 09:16",Как сделать QR код с картинкой на Python,http://habr.com/ru/post/652003/
329,"2022-02-17, 09:16",Уроки компьютерного зрения на Python + OpenCV ...,http://habr.com/ru/post/651989/
395,"2022-02-16, 09:37",Принципы проектирования SOLID с примерами на P...,http://habr.com/ru/company/otus/blog/651753/
421,"2022-02-15, 12:57",Реализация ряда Тейлора на Python,http://habr.com/ru/company/otus/blog/651659/


In [ ]:
# Задание 2.
# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]

# В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

# Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.


In [60]:
import requests
import pandas as pd

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

EMAILS =  ["wask-xxx@mail.ru", "bel@mail.ru"]
params = {'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
'Connection': 'keep-alive',
'Content-Length': '39',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/',
'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Microsoft Edge";v="98"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"Windows"',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.50',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Version': '0'}


df = pd.DataFrame(columns = ['дата утечки', 'источник утечки', 'описание утечки'])
for mail in EMAILS:
  number = 0
  r = requests.post(URL, headers=params, json= {'emailAddresses': [mail]}, verify=False)
  result_request = r.json()
  
  for key in result_request['data'].keys():
    breachId = result_request['data'][key][mail][0]['breachId']
    reason = result_request['breaches'][str(breachId)]['description']
    date = result_request['breaches'][str(breachId)]['publishDate']
    df.loc[mail+str(number)] = [date, key ,reason]

    number += 1


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [61]:
df

,дата утечки,источник утечки,описание утечки
wask-xxx@mail.ru0,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
wask-xxx@mail.ru1,2018-11-21T00:00:00Z,animoto.com,"In July 2018, the online video site Animoto su..."
wask-xxx@mail.ru2,2016-10-28T00:00:00Z,badoo.com,"In June 2016, millions of accounts compromised..."
wask-xxx@mail.ru3,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
wask-xxx@mail.ru4,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
wask-xxx@mail.ru5,2021-02-04T00:00:00Z,rzd-bonus.ru,"In June 2020, the Russian Railways Bonus Progr..."
bel@mail.ru0,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
bel@mail.ru1,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
bel@mail.ru2,2020-11-05T00:00:00Z,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
bel@mail.ru3,2020-12-10T00:00:00Z,nils.ru,"In November 2020, a collection of over 23,000 ..."
